In [2]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Basis State preparation for the two numbers
def state_preparation(basis_id, n_qubits):
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    return qml.BasisStatePreparation(bits, wires=range(n_qubits))
state_preparation(3,5)

BasisStatePreparation([0, 0, 0, 1, 1], wires=[0, 1, 2, 3, 4])

In [15]:
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    for i in range(int(n_qubits/2)):
        qml.SWAP(wires=[i,n_qubits-1-i])

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    n = n_qubits
    for i in range(n):
        qml.Hadamard(wires=i)
        for jj in range(i+1,n,1):
            qml.ControlledPhaseShift(np.pi/(2**(jj-i)), wires=[jj,i])
    pass

def qft_node(basis_id, basis_id2):
    s = basis_id + basis_id2
    n_qubits = len(np.binary_repr(s))
    print(n_qubits)
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    
    bits2 = [int(x) for x in np.binary_repr(basis_id2, width=n_qubits)]
    qml.BasisStatePreparation(bits2, wires=range(n_qubits, 2*(n_qubits),1))
    
    dev = qml.device('default.qubit', wires=(2*n_qubits), shots=10)
    @qml.qnode(dev) 
    def adder():
        qft_rotations(n_qubits)
        for i in range(0,n_qubits,1):
            k=0
            for j in range(i+n_qubits,(2*n_qubits),1):
                qml.ControlledPhaseShift((np.pi/(2**(k))), wires=[j,i])
                k+=1
        (qml.adjoint)(qft_rotations)(n_qubits)
        qml.draw(adder)()
    return qml.sample(wires=[x for x in range(n_qubits)])
print(qft_node(3,2))

3
sample(wires=[0, 1, 2])
